In [1]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import os

In [2]:
"""
entry_urls = []

start_date = "2006-07-02"
end_date = "2006-07-05"
#base_url = "https://rip.ie/death-notice/s/all?page={}&start={}+00%3A00%3A00&end={}&sortField=a.createdAtCastToDate&sortDir=DESC&view=list"
p_count = 1

for p in range(1, p_count+1):
    base_url = "https://rip.ie/death-notice/s/all?page=" + str(p) + "&start=" + start_date + "+00%3A00%3A00&end=" + end_date + "&sortField=a.createdAtCastToDate&sortDir=DESC&view=list"
    #entry_urls.append(base_url.format(p, start_date, end_date))
    entry_urls.append(base_url)
"""

'\nentry_urls = []\n\nstart_date = "2006-07-02"\nend_date = "2006-07-05"\n#base_url = "https://rip.ie/death-notice/s/all?page={}&start={}+00%3A00%3A00&end={}&sortField=a.createdAtCastToDate&sortDir=DESC&view=list"\np_count = 1\n\nfor p in range(1, p_count+1):\n    base_url = "https://rip.ie/death-notice/s/all?page=" + str(p) + "&start=" + start_date + "+00%3A00%3A00&end=" + end_date + "&sortField=a.createdAtCastToDate&sortDir=DESC&view=list"\n    #entry_urls.append(base_url.format(p, start_date, end_date))\n    entry_urls.append(base_url)\n'

In [3]:
def re_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        return None

In [4]:
def scrape_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [5]:
def get_url(soup):
    url_list = []
    anchors = soup.find_all('a', class_='showdown-dn-link')
    urls = [anchor['href'] for anchor in anchors]
    for url in urls:
        url_list.append("https://rip.ie" + url)
    return url_list

In [6]:
def get_ID(url):
    url_pattern = r'https://rip.ie/death-notice/([^/]+)-([^/]+)-([^/]+)(?:-([^/]+))?-(\d+)'
    m = re.match(url_pattern, url)
    if m:
        return m.group(5)  
    else:
        print("Pattern did not match.")

In [7]:
def get_name(soup):
    name_val = soup.find('h1', class_='DeathNotice_person-name__lkvex')
    if name_val:
        name = name_val.get_text()
        name = name.strip()
        name = name.replace("Â", "")
        name = name.replace("nee", "née")
        name = name.replace("nÃ©e", "née")
        name = name.replace("\n", " ")
        name = name.replace("â", "’")
        return name
    else:
        return None

In [8]:
def get_loc(soup):
    locations = {}
    loc_val = soup.find('div', class_='DeathNotice_tags-item__Fp1X4')
    if loc_val:
        raw_loc = loc_val.get_text()
        loc = [word.strip() for word in raw_loc.split(",") + raw_loc.split("/")]
        city_town = loc[-1]
        locations["City"] = city_town.split(",")[-1]
        locations["Town"] = city_town.split(",")[:-1]
        return locations
    else:
        return None

In [9]:
def get_dates(soup):
    date_list = {}

    published_date_value = soup.find('p', class_='DeathNotice_dates-published-date__M0A_i')
    if published_date_value:
        date_list["Published Date"] = published_date_value.get_text()
    else:
        date_list["Published Date"] = None

    death_date_value = soup.find('p', class_='DeathNotice_dates-death-date__bR7g_')
    if death_date_value:
        date_list["Death Date"] = death_date_value.get_text()
    else:
        date_list["Death Date"] = None

    return date_list

In [10]:
def get_obituary(soup):
    description = soup.find('div', class_='DeathNotice_description__sY_tC word-break')
    if description:
        text = description.get_text()
        text = text.strip()
        text = text.replace("Â", "")
        text = text.replace("nee", "née")
        text = text.replace("nÃ©e", "née")
        text = text.replace("\n", " ")
        text = text.replace("â", "’")
        text = text.replace("Å", "")
        text = text.replace("Ä", "")
        text = text.replace("", "")
        return text
    else:
        return None

In [11]:
def get_page_data(url_list):
    data_list = []
        
    for u in url_list:
        s = scrape_html(re_url(u))
        identity = get_ID(u)
        name = get_name(s)
        loc = get_loc(s)
        dates = get_dates(s)
        obituary_text = get_obituary(s)

        data_list.append({
            "Name": name,
            "ID": identity,
            "City": loc["City"],
            "Town": loc["Town"],
            "Published Date": dates["Published Date"],
            "Death Date": dates["Death Date"],
            "Obituary Text": obituary_text
        })

    df = pd.DataFrame(data_list)
    return df

In [12]:
def get_entry_urls(page_number):
    entry_urls = []
    #page_number = 40
    
    for p in range(1, page_number+1):
        base_url = "https://rip.ie/death-notice/s/all?page=" + str(p) + "&start=2006-07-02+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=ASC&view=list"
        entry_urls.append(base_url)
    return entry_urls

In [13]:
def get_url_list(entry_urls):
    url_list = []

    for url in entry_urls:
        page_data = re_url(url)
        if page_data:
            url_list.append(get_url(scrape_html(page_data)))
    return url_list

In [14]:
rip_df = get_page_data(get_url_list(get_entry_urls(2)))
rip_df

Error while processing ['https://rip.ie/death-notice/cornelius-con-ogorman-kerry-beaufort-533613', 'https://rip.ie/death-notice/cornelius-con-ogorman-kerry-lisselton-533613', 'https://rip.ie/death-notice/katalin-gaborne-baranya-tipperary-roscrea-533612', 'https://rip.ie/death-notice/paud-quinn-kerry-ventry-533611', 'https://rip.ie/death-notice/paud-quinn-kerry-ballyferriter-533611', 'https://rip.ie/death-notice/joseph-joe-walsh-galway-corofin-533610', 'https://rip.ie/death-notice/joseph-joe-walsh-offaly-tullamore-533610', 'https://rip.ie/death-notice/ishbel-mcgloughlin-wicklow-tinahely-533609', 'https://rip.ie/death-notice/nora-harrold-limerick-newcastle-west-533608', 'https://rip.ie/death-notice/ray-westby-waterford-dunmore-east-533607', 'https://rip.ie/death-notice/ray-westby-sligo-533607', 'https://rip.ie/death-notice/michael-mick-sweeney-kerry-tarbert-533606', 'https://rip.ie/death-notice/thomas-tom-moran-kildare-naas-533605', 'https://rip.ie/death-notice/thomas-tom-moran-dublin-du

TypeError: object of type 'NoneType' has no len()

In [ ]:
#output_directory = '/Users/idilbilgic/Desktop/STAGE4.1/COMP30170/suicide_rates_IE/output_monthly'
#rip_df.to_csv(os.path.join(output_directory, 'rip_output_antrim.csv'), index=False)

In [ ]:
#rip_df.to_csv('rip_output_2006.csv', index=False)